# SD201 TP Clustering 

In [4]:
import pandas as pd
import numpy as np
import random
import sklearn as sk

In [5]:
df = pd.read_csv("data.csv")
df.head()

,StockName,1/28/2011,4/29/2011,5/20/2011,4/1/2011,5/27/2011,6/17/2011,4/15/2011,2/18/2011,3/18/2011,...,1/14/2011,4/8/2011,4/21/2011,3/4/2011,3/25/2011,2/4/2011,1/7/2011,2/25/2011,5/13/2011,1/21/2011
0,American Express,-4.755700,4.00509,3.581550,-0.395257,0.768624,1.125940,-0.237274,-1.917280,0.706794,...,4.63801,1.468980,2.74809,-0.022868,1.877090,-0.70247,2.448040,-3.137520,-1.138630,-0.065175
1,Boeing,-3.201900,5.65488,-1.449280,0.693878,0.574788,1.505610,-1.425660,0.467675,-2.908530,...,0.93633,0.122649,3.74037,-0.924520,4.339170,3.06093,4.882840,-0.069109,-0.353045,1.157210
2,Chevron,-0.553840,1.92791,0.529256,1.804510,2.056760,-0.869652,-3.189360,3.371730,3.670840,...,2.06707,1.050500,3.03001,1.437230,2.811480,3.47363,-0.512765,2.892270,-0.832930,0.903809
3,Cisco Systems,0.431862,3.48494,-1.724140,-1.843320,0.304692,-1.122850,-3.839640,0.053079,-3.761930,...,1.28940,3.762490,0.35545,-1.181530,-0.346021,5.35117,2.542790,-0.480513,-3.707930,-2.356270
4,DuPont,3.819160,1.97522,0.000000,1.995930,1.565220,-0.919448,-1.009920,2.828800,-0.357277,...,3.10559,-0.180180,3.00295,-0.645518,0.557621,4.74576,-0.579421,-1.601460,-3.694940,-2.382390


In [6]:
def format(df:pd.DataFrame):
    ## Format data frame to numpy objects for computations
    df = df.drop(columns=["StockName"])
    data = df.to_numpy()
    return data


We choose this data structure for the graph and the clusters : 
- Each data point is a vertex denoted by a numpy array of coordinates.
- The clusters are represented by arrays of points belonging to the cluster for computation speed purposes.
- We keep a dictionnary with each vertex and its associated cluster (that we update when necessary).

We start by defining the function SSE.

In [47]:
def sse(data, centroid:dict):
    SSE = 0
    for i in range(len(data)):
        SSE += np.linalg.norm(data[i]- centroid[i])**2
    return SSE

In [65]:
def compute_centroids(clusters_list):
    k = len(clusters_list)
    new_centroids = []
    i = 0
    for cluster in clusters_list:
        print("Here is one cluster", cluster)

        ## Compute center of the cluster
        c = sum(np.array(cluster))/len(cluster)

        new_centroids.append(c)
        i+=1

    return new_centroids

In [75]:
def kmeans(data, k):
    
    ## We start by randomly selecting k points as centroids
    centroids = data[np.random.randint(data.shape[0], size=k), :]

    centroid = {}

    clusters_list = list([] for _ in range(k))

    ## Compute the closest centroid for every point in the data
    for p in range(len(data)):
        point = data[p]
        d_min = 10e99
        for i in range(len(centroids)):
            c = centroids[i]
            d = np.linalg.norm(c-point)
            if d < d_min:
                d_min = d
                closest_centroid = c
                best_cluster = i
        centroid[p] = c
        clusters_list[best_cluster].append(point)
    
    print("After the first step, the cluster list looks like this : ")
    print(clusters_list)
    
        
    
    SSE = sse(data, centroid)

    n_centroids = compute_centroids(clusters_list)

    while sum(np.linalg.norm(n_centroids[i] - centroids[i]) for i in range(len(centroids))) > 0:
        for point in data:
            d_min = 10e99
            for i in range(len(centroids)):
                c = centroids[i]
                d = np.linalg.norm(c-d)
                if d < d_min:
                    d_min = d
                    closest_centroid = c
                    best_cluster = i
                    centroid[i] = c
            clusters_list[best_cluster] = point
        
        centroids = n_centroids
        n_centroids = compute_centroids(clusters_list)

    return centroids, clusters_list  

In [76]:
data = format(df)
centroids, clusters_list = kmeans(data, 8)

After the first step, the cluster list looks like this : 
[[array([-0.55384 ,  1.92791 ,  0.529256,  1.80451 ,  2.05676 , -0.869652,
       -3.18936 ,  3.37173 ,  3.67084 , -4.0242  , -0.9911  , -0.853207,
       -1.21903 , -6.0285  , -3.45091 ,  2.06707 ,  1.0505  ,  3.03001 ,
        1.43723 ,  2.81148 ,  3.47363 , -0.512765,  2.89227 , -0.83293 ,
        0.903809]), array([ 3.20354 ,  5.64811 , -1.45461 ,  3.26821 ,  3.25765 , -1.01104 ,
       -2.55408 ,  2.22093 ,  2.40764 , -3.28757 ,  3.64805 ,  3.93495 ,
       -3.45137 , -5.07571 , -4.99013 ,  0.858277, -3.45495 ,  3.63705 ,
        0.311526,  2.04864 ,  3.59929 , -0.688705, -2.72745 , -4.0343  ,
       -1.49745 ]), array([ 1.63831 ,  0.354191, -4.35294 ,  1.98482 ,  3.25815 , -3.72793 ,
       -8.52713 , -0.632547,  1.00313 , -3.31725 ,  3.81731 ,  0.230814,
       -4.0201  , -0.694847, -4.8416  , -4.42849 ,  2.87026 ,  3.72861 ,
       -1.36823 ,  4.33455 ,  5.93325 ,  3.79267 , -1.76678 , -0.34965 ,
       -2.47066 ])], [ar

ZeroDivisionError: division by zero

In [87]:
import sklearn.cluster as clustering

clustering.k_means(data, 8)




(array([[-1.73819744e+00,  2.92546333e+00, -4.66360000e-01,
          6.15022444e-01,  1.07388423e+00,  6.13672200e-01,
         -1.30880622e+00,  5.43135644e-01, -1.75308289e+00,
         -8.29689678e-01, -1.16855016e+00,  2.79781956e+00,
         -1.10363244e+00, -1.11606156e+00, -4.01110778e+00,
          1.75756078e+00, -9.12143333e-03,  2.97348500e+00,
         -7.41405289e-01,  1.83650011e+00,  1.28860967e+00,
          1.57781107e+00, -1.21629417e+00, -1.04784878e+00,
          1.16091279e+00],
        [ 1.13288300e+00,  3.32450600e+00,  7.54920000e-03,
          1.57011680e+00,  2.40625200e+00, -6.25745200e-01,
         -1.75640872e+00,  2.68724600e+00,  1.69407460e+00,
         -2.94582000e+00,  1.03985080e+00,  7.53656000e-01,
         -2.10707200e+00, -4.64170200e+00, -3.43178500e+00,
          2.05932680e+00, -2.03377200e-01,  2.19444400e+00,
          7.85988600e-01,  1.54058380e+00,  4.42502600e+00,
          8.74311800e-01, -2.21424800e-01, -1.84810200e+00,
         -2.7